# 1.数据处理

## Step1.加载数据集（包含数据预处理） 


### 数据预处理


In [8]:
#数据预处理
from torchvision import transforms
# 查看定义数据预处理操作
import inspect
transform_methods = [name for name, obj in inspect.getmembers(transforms)
                     if inspect.isclass(obj) or inspect.isfunction(obj)]

# 打印结果
print(f"共找到 {len(transform_methods)} 个 transforms 方法：")
for name in sorted(transform_methods):
    print(name)

共找到 41 个 transforms 方法：
AugMix
AutoAugment
AutoAugmentPolicy
CenterCrop
ColorJitter
Compose
ConvertImageDtype
ElasticTransform
FiveCrop
GaussianBlur
Grayscale
InterpolationMode
Lambda
LinearTransformation
Normalize
PILToTensor
Pad
RandAugment
RandomAdjustSharpness
RandomAffine
RandomApply
RandomAutocontrast
RandomChoice
RandomCrop
RandomEqualize
RandomErasing
RandomGrayscale
RandomHorizontalFlip
RandomInvert
RandomOrder
RandomPerspective
RandomPosterize
RandomResizedCrop
RandomRotation
RandomSolarize
RandomVerticalFlip
Resize
TenCrop
ToPILImage
ToTensor
TrivialAugmentWide


上面这个单元格的输出体现了Transform有很多种**对于数据进行预处理**的方法

你也可以理解为Transform可以对数据进行增强

不过我们一般来说,使用compose方法可以交叉使用多种方法

其实常见的就那几种:剪裁,放大,随机翻转,以及crop

然后,ToTensor和Normalize方法是必须的,进行向量化和归一化

In [9]:
from torchvision import transforms
##用法,常见套路
transform = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),#常用的归一化参数
])

OK定义完数据处理的方法之后

我们就可以进行加载数据集了

这里有两种方法,一种是加载标准数据集,一种是加载自定义数据集

### 方法一：加载标准数据集
标准数据集就是torch库里面自带的数据集

In [1]:
from torchvision import datasets
import inspect
datasets_classes = [name for name,obj in inspect.getmembers(datasets)if inspect.isclass(obj) and issubclass(obj,datasets.VisionDataset)]
print(f"共找到 {len(datasets_classes)} 个数据集：")
for name in sorted(datasets_classes):
    print(name)


共找到 72 个数据集：
CIFAR10
CIFAR100
CLEVRClassification
CREStereo
Caltech101
Caltech256
CarlaStereo
CelebA
Cityscapes
CocoCaptions
CocoDetection
Country211
DTD
DatasetFolder
EMNIST
ETH3DStereo
EuroSAT
FER2013
FGVCAircraft
FakeData
FallingThingsStereo
FashionMNIST
Flickr30k
Flickr8k
Flowers102
FlyingChairs
FlyingThings3D
Food101
GTSRB
HD1K
HMDB51
INaturalist
ImageFolder
ImageNet
Imagenette
InStereo2k
KMNIST
Kinetics
Kitti
Kitti2012Stereo
Kitti2015Stereo
KittiFlow
LFWPairs
LFWPeople
LSUN
LSUNClass
MNIST
Middlebury2014Stereo
MovingMNIST
Omniglot
OxfordIIITPet
PCAM
PhotoTour
Places365
QMNIST
RenderedSST2
SBDataset
SBU
SEMEION
STL10
SUN397
SVHN
SceneFlowStereo
Sintel
SintelStereo
StanfordCars
UCF101
USPS
VOCDetection
VOCSegmentation
VisionDataset
WIDERFace


上面的单元格输出体现了torch库里自带了多种数据集
下面是如何一键下载这些数据集

In [5]:
#用法 ,下载这个datasets的标准数据集,
dataset=datasets.CIFAR10(root='./测试datasets库里的标准数据集',train=True,transform=transform,download=True)

#dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)

100.0%


Extracting ./测试datasets库里的标准数据集\cifar-10-python.tar.gz to ./测试datasets库里的标准数据集


但是我们很多情况下都是有自己的数据集,那该怎么对文件夹进行加载呢?

pytorch提供了一个Dataset的抽象类,我们只需继承这个抽象类

然后在里面定义3个方法就行了.(from torch.utils.data import Dataset)这样导入抽象类

### 方法二 加载自定义数据集

In [37]:
from torch.utils.data import Dataset
import os
from PIL import Image
import torch
import numpy as np 
#自定义数据集类,只需要重写三种方法
#__init__初始化,__len__获取长度,以及__getitem__进行对象的切片

#假设对文件夹train_and_label进行自定义类读取
#该文件夹有两个子集,img是特征值,label是标签值
class SegmentationDataset(Dataset):
  
  #1.第一个方法,初始化参数和方法
  def __init__(self, img_dir, label_dir, transform=None) -> None:
    super().__init__()
    self.img_dir = img_dir  # 存储图像文件夹路径
    self.label_dir = label_dir  # 存储标签文件夹路径
    self.transform = transform  # 数据增强操作
    self.img_names = os.listdir(img_dir)  # 获取图像文件名,将文件名存在列表img_names里
    
    
  #2. 第二个方法,返回数据的大小
  def __len__(self):
    return len(self.img_names)
    pass
  #3. 第三个方法,如何读取数据,如何进行数据增强
  def __getitem__(self,idx):
    img_name=self.img_names[idx]
    img_path=os.path.join(self.img_dir,img_name) #进行单个数据的名称的路径合并
    label_name=img_name.split('.')[0]+'.png'     #通过split,其'.'前面部分拼接上后缀
    label_path=os.path.join(self.label_dir,label_name)
    
    #接下来打开图像
    img=Image.open(img_path).convert('RGB')
    label=Image.open(label_path).convert('L')   #因为标签值是语义分割,所以就是要转化为灰度图
    
    #判断一下有没有数据增强的方法
    if(self.transform):
      img=self.transform(img)      #一般img是输入,可以随便数据增强,
      label = torch.tensor(np.array(label), dtype=torch.long)  #标签值就不能Transform数据增强了,转个向量就行
    
    return img,label #返回一下,数据集的切片返回特征值img和标签值label

#测试一下
dataset=SegmentationDataset(
  img_dir='./train_and_label/img',
  label_dir='./train_and_label/label',
  transform=transform
)
print(f"数据集长度: {len(dataset)}")#输出3,没问题!

数据集长度: 3


In [6]:
#测试一下如何读取文件夹里的文件名?
#用os.listdir()函数
import os
img_name=os.listdir("train_and_label/img")
img_name

['T000000.jpg', 'T000002.jpg', 'T000003.jpg']

In [45]:
from torch.utils.data import Dataset
import os
from PIL import Image
import torch
import numpy as np 
class SegmentationDataset(Dataset):
    def __init__(self, img_dir, label_dir, transforms=None):
        self.img_dir = img_dir
        self.label_dir = label_dir
        self.transform = transform
        self.img_names = os.listdir(img_dir)  # 获取图像文件名

    def __len__(self):
        return len(self.img_names)

    def __getitem__(self, idx):
        img_name = self.img_names[idx] # idx = 0
        img_path = os.path.join(self.img_dir, img_name)
        
        label_name = os.path.splitext(img_name)[0] + ".png"
        label_path = os.path.join(self.label_dir, label_name)

        # 加载图片和标签
        image = Image.open(img_path).convert("RGB")  # 转换为 RGB 图像
        label = Image.open(label_path).convert("L")  # 转换为单通道图像

        if self.transform:
            image = self.transform(image)
            label = torch.tensor(np.array(label), dtype=torch.long)  # 转换为张量
        
        return image, label
data = SegmentationDataset(
    img_dir='train_and_label/img',
    label_dir='train_and_label/label',
    transforms=transform)

接下来我们来看数据集的划分


## Step2.数据集划分

In [46]:
from torch.utils.data import random_split
train_size = int(0.8 * len(dataset))
val_size = int(0.1 * len(dataset))
test_size = len(dataset) - train_size - val_size
train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size])


## Step3.数据集读取

In [47]:
from torch.utils.data import DataLoader
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# 2.模型搭建

## 方法一 容器搭建
适用于比较简单,层数比较小的模型

In [48]:
from torch import nn
#model = nn.Sequential(nn.Linear(784, 256), nn.ReLU(), nn.Linear(256, 10))
model=nn.Sequential(
  nn.Linear(32*32*3, 256),  # 假设输入图像大小为32x32，3个通道,256是中间隐藏层的神经元数量
  nn.ReLU(),
  nn.Linear(256, 10)  # 假设输出是10个类别
)
print(model)

Sequential(
  (0): Linear(in_features=3072, out_features=256, bias=True)
  (1): ReLU()
  (2): Linear(in_features=256, out_features=10, bias=True)
)


## 方法二 自定义模型
模型比较复杂

重写一个nn中的Module抽象类


In [49]:
from torch import nn
from torch.nn import Module
class MLP(Module):
   
    def __init__(self, input_size,hidden_size,output_size) -> None:
        #super(MLP,self).__init__()
        super().__init__()
        self.fc1=nn.Linear(input_size,hidden_size)
        self.relu=nn.ReLU()
        self.fc2=nn.Linear(hidden_size,output_size)
        
    def forward(self,x):
        x=self.fc1(x)
        x=self.relu(x)
        x=self.fc2(x)
        return x
model=MLP(input_size=784,hidden_size=256,output_size=10)
print(model)       
   


MLP(
  (fc1): Linear(in_features=784, out_features=256, bias=True)
  (relu): ReLU()
  (fc2): Linear(in_features=256, out_features=10, bias=True)
)


In [32]:
from torch.nn import Module
class MLP(Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x
model = MLP(input_size=784, hidden_size=256, output_size=10)

# 3.损失函数

In [50]:
from torch.nn import CrossEntropyLoss
criterion = CrossEntropyLoss()

# 4.优化器

In [51]:
from torch import optim
optimizer = optim.Adam(model.parameters(), lr=0.001)


# 5.模型训练

In [52]:
num_epochs = 10
for epoch in range(num_epochs):
    model.train()  # 设置模型为训练模式
    loss_all = 0.0
    for inputs,labels in train_dataloader:
        outputs=model(inputs)
        loss_value=criterion(outputs,labels)
        optimizer.zero_grad()  # 清除梯度
        loss_value.backward()
        optimizer.step()
        loss_all += loss_value.item() * inputs.size(0)
    epoch_loss = loss_all / len(train_dataset)

RuntimeError: mat1 and mat2 shapes cannot be multiplied (192x32 and 784x256)

In [36]:
num_epochs = 5  # 设置训练的轮数
for epoch in range(num_epochs):
    model.train()  # 设置模型为训练模式
    running_loss = 0.0
    correct = 0
    total = 0
    for inputs, labels in train_dataloader:        
        # 前向传播
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        
        # 反向传播和优化
        optimizer.zero_grad()  # 清空梯度
        loss.backward()        # 计算梯度
        optimizer.step()       # 更新权重
        
        running_loss += loss.item()
        
        # 计算准确率
        _, predicted = torch.max(outputs, 1)  # 选择最大概率的类别
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    epoch_loss = running_loss / len(train_dataloader)
    epoch_acc = 100 * correct / total
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}, Accuracy: {epoch_acc:.2f}%")


RuntimeError: mat1 and mat2 shapes cannot be multiplied (192x32 and 784x256)

In [43]:
print(inputs.shape)


torch.Size([2, 3, 32, 32])
